<a href="https://colab.research.google.com/github/DonghaeSuh/NLP_Pytorch/blob/main/Model/GPT_2/GPT2_GEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentencepiece : https://github.com/google/sentencepiece/blob/master/python/README.md

### NSMC

In [229]:
import urllib

In [230]:
urllib.request.urlretrieve('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt','train.txt')

('train.txt', <http.client.HTTPMessage at 0x77fbd5b64850>)

In [231]:
temp=[]

In [232]:
with open('train.txt','r') as f:
  f.readline()
  for sent in f.readlines():
    data = sent.split('\t')
    temp.append(data[1])

In [226]:
cd drive/MyDrive/Pytorch\ NLP/GPT-2/data_in/gpt2_ckpt

/content/drive/MyDrive/Pytorch NLP/GPT-2/data_in/gpt2_ckpt


In [233]:
with open('corpus.txt','w') as f:
  for sent in temp:
    f.write(f'\n {sent}')

### Sentencepiece

https://keep-steady.tistory.com/7

In [212]:
pip install sentencepiece

In [234]:
import sentencepiece as spm

In [235]:
 spm.SentencePieceTrainer.train(input='corpus.txt',model_prefix='kor_tokenizer',vocab_size=50257, model_type='bpe',max_sentence_length=9999999,pad_id=0,unk_id=1,bos_id=2,eos_id=3)

### gluonnlp

In [237]:
sp=spm.SentencePieceProcessor()
sp.Load('kor_tokenizer.model')

True

In [238]:
sp.EncodeAsPieces('안녕하세요 저는 사람이에요.')

['▁안녕하세요', '▁저는', '▁사람이', '에요', '.']

In [239]:
sp.encode('안녕하세요 저는 사람이에요.')

[20363, 1619, 1047, 802, 48546]

In [240]:
sp.encode('그래')

[312]

In [241]:
sp.bos_id()

2

In [242]:
vocab = [sp.id_to_piece(id) for id in range(sp.get_piece_size())]

In [243]:
print('0 :{}, 1 :{} , 2 :{}, 3 :{} '.format(vocab[0],vocab[1],vocab[2],vocab[3]))

0 :<pad>, 1 :<unk> , 2 :<s>, 3 :</s> 


### config & model_weight

In [ ]:
urllib.request.urlretrieve('https://huggingface.co/gpt2/resolve/main/config.json','config.json')

('config.json', <http.client.HTTPMessage at 0x7fdd07c1ada0>)

In [ ]:
urllib.request.urlretrieve('https://huggingface.co/gpt2/resolve/main/model.safetensors','model.safetensors')

('model.safetensors', <http.client.HTTPMessage at 0x7fdd07c19ed0>)

## Top k & Top p

### torch.multinomial

In [94]:
import torch
import torch.nn as nn

In [193]:
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
  # logits = (vocab_size, )

  if top_k>0:
    top_k=min(top_k,logits.size()[0])
    indices_to_remove=logits<torch.topk(logits,top_k).values[-1] # smaller than the smallest value among the top_k values
    logits[indices_to_remove]=filter_value

  if top_p>0.0:
    sorted_logits=torch.sort(logits,descending=True)
    sorted_logits_index=torch.argsort(logits,descending=True)
    prob_cumsum = torch.cumsum(nn.Softmax()(sorted_logits))

    sorted_indices_to_remove=prob_cumsum>top_p
    sorted_indices_to_remove=torch.cat([torch.LongTensor([False]),sorted_indices_to_remove[:-1]]) # prevent if first is True
    indices_to_remove = sorted_logits_index[sorted_indices_to_remove]

    logits[indices_to_remove]=filter_value

  return logits   # logits = (vocab_size, )


## Greedy & Generate_sentence Function

In [244]:
def generate_sent(seed_word,model,max_len=100,greedy=False,top_k=0,top_p=0.0):
  sent= seed_word
  toked = sp.encode(sent)

  for _ in range(max_len):
    input_ids = torch.LongTensor([sp.bos_id()]+toked)[None,:] # input_ids = (1, cumulated_seq_len)
    outputs = model(input_ids)[0,-1,:] # outputs : (vocab_size, )

    if greedy:
      gen = sp.id_to_piece(outputs.argmax().tolist()) # outputs.argmax().tolist() -> int
    else:
      output_logits = top_k_top_p_filtering(outputs,top_k,top_p) # logits = (vocab_size, )
      gen = torch.multinomial(nn.Softmax(-1)(output_logits),num_samples=1,replacement=True).tolist()[0]
      gen = sp.id_to_piece(gen)
    if gen == '</s>':
      break

    sent+=gen.replace('▁',' ')
    toked=sp.encode(sent)

  return sent

## Huggingface Transformers

In [175]:
ls

config.json  kor_tokenizer.model  model.safetensors
corpus.txt   kor_tokenizer.vocab


In [ ]:
pip install transformers

In [178]:
from transformers import GPT2LMHeadModel

### Model

In [180]:
cd ..

/content/drive/MyDrive/Pytorch NLP/GPT-2


In [187]:
class GPT2Gen(nn.Module):
  def __init__(self,dir_path):
    super(GPT2Gen,self).__init__()
    self.gpt2 = GPT2LMHeadModel.from_pretrained(dir_path,ignore_mismatched_sizes=True)

  def forward(self,x):
    return self.gpt2(x)[0] # (batch_size=1, cumulated_seq_len, vocab_size =500000)

In [ ]:
gpt2_model = GPT2Gen('./data_in/gpt2_ckpt')

Generate_sentences

In [196]:
generate_sent('안녕',gpt2_model,top_k=10)

<ipython-input-195-c286d1860d07>:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  gen = torch.multinomial(nn.Softmax()(output_logits),num_samples=1,replacement=True).tolist()[0]


'안녕 행복하길 바쁘노릇실상 귀여트랑실상 협주곡실상실상 협주곡 귀여태격태격태격 행복하길 있었는지 귀여실상실상 귀여 협주곡 무료영화 니가 귀여실상 귀여트랑 무료영화트랑트랑태격 귀여실상실상 귀여딩딩태격 니가 협주곡트랑태격태격실상 무료영화실상실상실상실상트랑 협주곡실상 바쁘 니가 가고싶실상실상실상 바쁘트랑태격 협주곡실상실상 기대되네요트랑실상실상 협주곡실상실상실상실상실상실상높음 협주곡실상 가고싶 바쁘 단어에실상트랑실상 단어에 짜맞추 협주곡실상 단어에트랑트랑 협주곡 협주곡 바쁘트랑트랑 짜맞추실상 바쁘'